In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Global Health and Nutrition
### The World Bank: an international financial institution that provides loans to countries of the world for capital projects

Will use BigQuery dataset for interactive visualizations. Will see what is the average age of first marriages for females around the world. For more details, please visit <a href='https://www.kaggle.com/theworldbank/world-bank-health-population'>here</a>

#### Import libraries

In [15]:
!pip install chart_studio

In [ ]:
!pip install plotly --upgrade

Requirement already up-to-date: plotly in /opt/conda/lib/python3.6/site-packages (4.3.0)


In [ ]:
import bq_helper
from google.cloud import bigquery
from bq_helper import BigQueryHelper

ghnp = bq_helper.BigQueryHelper(active_project='bigquery-public-data', dataset_name='world_bank_health_population')
bq_assistant = BigQueryHelper('bigquery-public-data', 'world_bank_health_population')
bq_assistant.list_tables()

In [ ]:
bq_assistant.head('health_nutrition_population', num_rows=5)

### Average age of first marriages for females around the world since 2000

In [ ]:
qr_averageFirstMarriage = """
select country_name, country_code, round(avg(value), 2) as average
from `bigquery-public-data.world_bank_health_population.health_nutrition_population`
where indicator_code = 'SP.DYN.SMAM.FE' and year > 2000
group by country_name, country_code
order by average
"""

averageFirstMarriage = ghnp.query_to_pandas_safe(qr_averageFirstMarriage)
averageFirstMarriage.head()

In [ ]:
# Use iplot()
# See Details on iplot: https://kite.com/python/docs/plotly.offline.iplot
# It plotly graphs inside an IPython notebook without connecting to an external server

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()
from subprocess import check_output

data = [{
    'type': 'choropleth',
    'locations': averageFirstMarriage.country_code,
    'z': averageFirstMarriage.average,
    'text': averageFirstMarriage.country_name,
    'autocolorscale': True,
    'reversescale': True,
    'marker':{
        'line':{
            'color': 'rgb(180,180,180)',
            'width': 0.5
        }
    }
}]


layout={
    'title': 'Average Age of First Marriage of Females Since 2000',
    'geo': {
        'showframe': False,
        'showcoastlines': False,
        'projection': {'type': 'orthographic'}
    }
}

figure={'data': data, 'layout': layout}

# Show a Plot
iplot(figure)

### Life Expectancy

In [ ]:
qr_lifeExpectancy = """
select country_name, country_code, year, round(value, 2) as indicator
from `bigquery-public-data.world_bank_health_population.health_nutrition_population`
where indicator_code = 'SP.DYN.LE00.IN'
order by year;
"""

lifeExpectancy = ghnp.query_to_pandas_safe(qr_lifeExpectancy)
lifeExpectancy.head()

In [ ]:
# https://plot.ly/python/animations/

def plotIndicatorMapOverTime(df,title):
    gDataList = []
    frames = []
    years = df.year.unique().tolist()

    #Create template Figure
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }

    figure['layout']['title'] = title
    
    #Define Sliders
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 400,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': str(years[0]),
        'plotlycommand': 'animate',
        'values': years,
        'visible': True
    }

    #Update buttons: Play and Pause
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration':0, 'redraw': False},
                             'fromcurrent': True, 'transition': {'duration': 0, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]
    
    sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
    }
    
    minZValue = df.indicator.min()
    maxZValue = df.indicator.max()
    
    for year in years:
        dfYear = df[df.year == year]
        gDataTemp = [ {
            'type': 'choropleth',
            'locations': dfYear.country_code,
            'z': dfYear.indicator,
            'text': dfYear.country_name,
            'autocolorscale': True,
            'marker': {
                'line': {
                    'color':'rgb(180,180,180)',
                    'width':0.5
                        } 
                      },
            'zauto': False,
            'zmin': minZValue,
            'zmax': maxZValue,
        } ]
    
        if year == years[0]:     
            figure['data'] = gDataTemp

    
        frame = {'data': gDataTemp,'name': str(year)}
        figure['frames'].append(frame)
    
        slider_step = {'args': [
            [year],
            {'frame': {'duration': 300, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 300}}
         ],
         'label': year,
         'method': 'animate'}
        
        sliders_dict['steps'].append(slider_step)
    
    figure['layout']['sliders'] = [sliders_dict]

    iplot(figure)

In [ ]:
plotIndicatorMapOverTime(lifeExpectancy, title='Life Expectancy')